# Crime in Chicago

The objective of this project is to predict whether a person who committed a particular crime was arrested.

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine  
import pickle
import feather
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# This is the local data path
#data_path = '/Users/kevin/Downloads/Crimes_-_2001_to_paresent.csv'

In [3]:
weather_data_path = '/Users/kevin/Downloads/1598904.csv'

In [41]:
iucr_codes = '/Users/kevin/Downloads/Chicago_Police_Department_-_Illinois_Uniform_Crime_Reporting__IUCR__Codes.csv'

/Users/kevin/Downloads/Chicago_Police_Department_-_Illinois_Uniform_Crime_Reporting__IUCR__Codes.csv

CREATE TABLE IF NOT EXISTS ChicagoCrime (
        ID integer,
        CaseNumber varchar(20),
        Date varchar(50),
        Block varchar(50),
        IUCR varchar(10),
        PrimaryType varchar(50),
        Description varchar(100),
        LocationDescription varchar(150),
        Arrest varchar(10),
        Domestic varchar(10),
        Beat integer,
        District real,
        Ward real,
        CommunityArea real,
        FBICode varchar(10),
        XCoordinate varchar(20),
        YCoordinate varchar(20),
        Year integer,
        UpdatedOn varchar(50),
        Latitude varchar(15),
        Longitude varchar(15),
        Location varchar(50)
    );


## Pull Data From Server

In [13]:
cnx = create_engine('postgresql://ubuntu:@54.91.118.64:5432/chicago')

In [14]:
df = pd.read_sql_query('''SELECT * FROM chicagocrime''', cnx)

OperationalError: (psycopg2.OperationalError) could not connect to server: Operation timed out
	Is the server running on host "54.91.118.64" and accepting
	TCP/IP connections on port 5432?
 (Background on this error at: http://sqlalche.me/e/e3q8)

In [29]:
df.head()

,id,casenumber,date,block,iucr,primarytype,description,locationdescription,arrest,domestic,...,ward,communityarea,fbicode,xcoordinate,ycoordinate,year,updatedon,latitude,longitude,location
0,10000092,HY189866,03/18/2015 07:44:00 PM,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,false,false,...,28.0,25.0,04B,1144606,1903566,2015,02/10/2018 03:50:01 PM,41.891398861,-87.744384567,"(41.891398861, -87.744384567)"
1,10000094,HY190059,03/18/2015 11:00:00 PM,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,true,false,...,15.0,67.0,26,1166468,1860715,2015,02/10/2018 03:50:01 PM,41.773371528,-87.665319468,"(41.773371528, -87.665319468)"
2,10000095,HY190052,03/18/2015 10:45:00 PM,044XX S LAKE PARK AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,true,...,4.0,39.0,08B,1185075,1875622,2015,02/10/2018 03:50:01 PM,41.81386068,-87.596642837,"(41.81386068, -87.596642837)"
3,10000096,HY190054,03/18/2015 10:30:00 PM,051XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,APARTMENT,false,false,...,3.0,40.0,08B,1178033,1870804,2015,02/10/2018 03:50:01 PM,41.800802415,-87.622619343,"(41.800802415, -87.622619343)"
4,10000097,HY189976,03/18/2015 09:00:00 PM,047XX W ADAMS ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,false,false,...,28.0,25.0,03,1144920,1898709,2015,02/10/2018 03:50:01 PM,41.878064761,-87.743354013,"(41.878064761, -87.743354013)"


In [38]:
df['datetime'] = pd.to_datetime(df['date'], infer_datetime_format=True)

In [ ]:
mask = df['arrest']  == 'true'
print('Percent of Crimes ending in Arrest: ' + len(df[mask])/len(df)

In [42]:
#df.to_feather('chicago_crime.feather')

In [95]:
df = feather.read_dataframe('chicago_crime')

In [96]:
df.head()

,id,casenumber,date,block,iucr,primarytype,description,locationdescription,arrest,domestic,...,communityarea,fbicode,xcoordinate,ycoordinate,year,updatedon,latitude,longitude,location,datetime
0,10000092,HY189866,03/18/2015 07:44:00 PM,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,false,false,...,25.0,04B,1144606,1903566,2015,02/10/2018 03:50:01 PM,41.891398861,-87.744384567,"(41.891398861, -87.744384567)",2015-03-18 19:44:00
1,10000094,HY190059,03/18/2015 11:00:00 PM,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,true,false,...,67.0,26,1166468,1860715,2015,02/10/2018 03:50:01 PM,41.773371528,-87.665319468,"(41.773371528, -87.665319468)",2015-03-18 23:00:00
2,10000095,HY190052,03/18/2015 10:45:00 PM,044XX S LAKE PARK AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,true,...,39.0,08B,1185075,1875622,2015,02/10/2018 03:50:01 PM,41.81386068,-87.596642837,"(41.81386068, -87.596642837)",2015-03-18 22:45:00
3,10000096,HY190054,03/18/2015 10:30:00 PM,051XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,APARTMENT,false,false,...,40.0,08B,1178033,1870804,2015,02/10/2018 03:50:01 PM,41.800802415,-87.622619343,"(41.800802415, -87.622619343)",2015-03-18 22:30:00
4,10000097,HY189976,03/18/2015 09:00:00 PM,047XX W ADAMS ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,false,false,...,25.0,03,1144920,1898709,2015,02/10/2018 03:50:01 PM,41.878064761,-87.743354013,"(41.878064761, -87.743354013)",2015-03-18 21:00:00


You have the IUCR codes so pull in the description for those codes in your downloads

Can check Chicago Economy for more data

## Read in Weather Data

In [97]:
df_weather = pd.read_csv(weather_data_path)

WT03 - Thunder  
WT04 - Ice pellets, sleet, snow pellets, or small hail"  
PRCP - Precipitation  
WT05 - Hail (may include small hail)  
WV03 - Thunder  
WT06 - Glaze or rime   
WT07 - Dust, volcanic ash, blowing dust, blowing sand, or blowing obstruction  
WT08 - Smoke or haze   
SNWD - Snow depth  
WT09 - Blowing or drifting snow  
WDF2 - Direction of fastest 2-minute wind  
WDF5 - Direction of fastest 5-second wind  
PGTM - Peak gust time  
WT11 - High or damaging winds  
TMAX - Maximum temperature  
WT13 - Mist  
WSF2 - Fastest 2-minute wind speed  
FMTM - Time of fastest mile or fastest 1-minute wind  
WSF5 - Fastest 5-second wind speed  
SNOW - Snowfall  
WT14 - Drizzle  
WT15 - Freezing drizzle   
WT16 - Rain (may include freezing rain, drizzle, and freezing drizzle)"   
WT17 - Freezing rain   
WT18 - Snow, snow pellets, snow grains, or ice crystals  
WT19 - Unknown source of precipitation   
AWND - Average wind speed  
WT21 - Ground fog  
WT22 - Ice fog or freezing fog  
WV20 - Rain or snow shower  
WT01 - Fog, ice fog, or freezing fog (may include heavy fog)  
WESD - Water equivalent of snow on the ground  
WT02 - Heavy fog or heaving freezing fog (not always distinguished from fog)  
TAVG - Average Temperature.  
TMIN - Minimum temperature  
TSUN - Total sunshine for the period  

In [98]:
df_weather.columns = map(str.lower, df_weather.columns)

In [99]:
df_weather['datetime'] = pd.to_datetime(df_weather['date'], infer_datetime_format=True)

## Merge Weather Data and Crime Data

In [100]:
df_weather = df_weather.sort_values('datetime')

In [101]:
df = df.sort_values('datetime')

In [102]:
cw_df = pd.merge_asof(df, df_weather, on = 'datetime', direction = 'backward', tolerance = pd.Timedelta('1 day')) 


In [103]:
#iucr_df = pd.read_csv(iucr_codes)

In [104]:
#df = cw_df.set_index('iucr').join(iucr_df.set_index('IUCR'))

In [105]:
df = cw_df.reset_index()

In [106]:
df.to_feather('chicago_crime_and_weather.feather')

## View the Entire Dataset

In [149]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [244]:
display_all(df)


,primarytype,description,locationdescription,arrest,domestic,beat,district,ward,communityarea,fbicode,latitude,longitude,awnd,prcp,snow,tmax,tmin
1,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,0,0,511,5.0,6.0,49.0,02,41.715923,-87.614138,7.61,0.00,0.0,24,5
2,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,0,0,512,5.0,34.0,49.0,02,41.702605,-87.632285,7.61,0.00,0.0,24,5
3,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,0,0,1013,10.0,22.0,30.0,26,41.847059,-87.727795,7.61,0.00,0.0,24,5
4,OFFENSE INVOLVING CHILDREN,SEX ASSLT OF CHILD BY FAM MBR,RESIDENCE,1,0,813,8.0,13.0,62.0,02,41.791519,-87.729099,7.61,0.00,0.0,24,5
5,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,0,0,1033,10.0,12.0,30.0,06,41.848636,-87.700117,7.61,0.00,0.0,24,5
6,DECEPTIVE PRACTICE,FORGERY,COMMERCIAL / BUSINESS OFFICE,0,0,2213,22.0,19.0,72.0,10,41.720449,-87.681904,7.61,0.00,0.0,24,5
7,THEFT,OVER $500,RESIDENCE,0,0,2124,2.0,4.0,39.0,06,41.805232,-87.596903,7.61,0.00,0.0,24,5
8,ASSAULT,SIMPLE,RESIDENCE,0,0,2535,25.0,1.0,23.0,08A,41.903598,-87.724043,7.61,0.00,0.0,24,5
9,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,0,0,512,5.0,9.0,49.0,06,41.700687,-87.619501,7.61,0.00,0.0,24,5
10,OFFENSE INVOLVING CHILDREN,CRIM SEX ABUSE BY FAM MEMBER,RESIDENCE,0,1,825,8.0,15.0,66.0,20,41.777017,-87.686172,7.61,0.00,0.0,24,5


In [112]:
df = df.drop('index', axis = 1)

In [113]:
df = df.drop('casenumber', axis = 1)

In [114]:
df = df.drop('id', axis = 1)

In [115]:
df = df.drop('block', axis = 1)

In [116]:
df['primarytype'] = df['primarytype'].astype('category')

In [117]:
df['description'] = df['description'].astype('category')

In [118]:
df['locationdescription'] = df['locationdescription'].astype('category')

In [120]:
#df['arrest'] = df['arrest'].astype('int')
df['arrest'].replace('true', 1, inplace = True)
df['arrest'].replace('false', 0, inplace = True)

In [122]:
#df['domestic'] = df['domestic'].astype('bool')
df['domestic'].replace('true', 1, inplace = True)
df['domestic'].replace('false', 0, inplace = True)

In [123]:
df['fbicode'] = df['fbicode'].astype('category')

In [124]:
df = df.drop('date_x', axis = 1)

In [125]:
df['xcoordinate'] = df['xcoordinate'].fillna(value=np.nan)

In [126]:
df['xcoordinate'] = df['xcoordinate'].astype('int64', errors = 'ignore')

In [127]:
df['ycoordinate'] = df['ycoordinate'].fillna(value=np.nan)

In [128]:
df['ycoordinate'] = df['ycoordinate'].astype('int64', errors = 'ignore')

In [129]:
df = df.drop('year', axis = 1)

In [130]:
df = df.drop('updatedon', axis = 1)

In [131]:
df['latitude'] = df['latitude'].astype('float64', errors = 'ignore')

In [132]:
df['longitude'] = df['longitude'].astype('float64', errors = 'ignore')

In [133]:
df = df.drop('location', axis = 1)

In [134]:
df['station'] = df['station'].astype('category')

In [135]:
df = df.drop('name', axis = 1)

In [136]:
df = df.drop('date_y', axis = 1)

In [137]:
df = df.drop('iucr', axis = 1)

In [141]:
df.to_feather('chicago_crime_cleaned.feather')

In [208]:
df = feather.read_dataframe('chicago_crime_cleaned.feather')

In [209]:
df = df.drop(['wdf2', 'wdf5', 'wesd', 'wsf2', 'wsf5', 'wt01',
       'wt02', 'wt03', 'wt04', 'wt05', 'wt06', 'wt07', 'wt08', 'wt09', 'wt11',
       'wt13', 'wt14', 'wt15', 'wt16', 'wt17', 'wt18', 'wt19', 'wt21', 'wt22',
       'wv03', 'wv20', 'tsun'], axis = 1)

In [210]:
df = df.drop('station', axis = 1)

In [225]:
df = df.drop('fmtm', axis = 1)

In [226]:
df = df.drop('pgtm', axis = 1)

In [229]:
df = df.drop('snwd', axis = 1)

In [211]:
df = df.drop('xcoordinate', axis = 1)
df = df.drop('ycoordinate', axis = 1)

In [212]:
df = df.drop('datetime', axis = 1)

In [ ]:
df = df.dropna(subset=['district'])

In [232]:
df = df.drop('tavg', axis = 1)

In [234]:
df = df.dropna(subset=['latitude'])

In [220]:
df['locationdescription'] = df.locationdescription.fillna(value='OTHER')

In [221]:
df['communityarea'] = df.sort_values(by=['beat', 'district', 'ward'])['communityarea'].fillna(method='ffill')

In [223]:
df['ward'] = df.sort_values(by=['beat', 'district', 'communityarea'])['ward'].fillna(method='ffill')

In [ ]:
for header in df.columns:
    
    nulls_count = df[f'{header}'].isnull().sum()
    
    print(f'There are {nulls_count} in {header}')

## EDA

In [248]:
df = df.reset_index()

In [249]:
df.to_feather('chicago_crime_final.feather')

In [2]:
df = feather.read_dataframe('chicago_crime_final.feather')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6735796 entries, 0 to 6735795
Data columns (total 18 columns):
index                  int64
primarytype            category
description            category
locationdescription    category
arrest                 int64
domestic               int64
beat                   int64
district               float64
ward                   float64
communityarea          float64
fbicode                category
latitude               float64
longitude              float64
awnd                   float64
prcp                   float64
snow                   float64
tmax                   int64
tmin                   int64
dtypes: category(4), float64(8), int64(6)
memory usage: 758.0 MB


In [4]:
df.corr()

,index,arrest,domestic,beat,district,ward,communityarea,latitude,longitude,awnd,prcp,snow,tmax,tmin
index,1.000000,-0.055044,0.043370,-0.035996,-0.004956,0.013127,0.004968,-0.005265,0.001056,0.036246,0.016684,0.020514,0.017340,0.038502
arrest,-0.055044,1.000000,-0.069274,-0.015993,-0.016780,-0.015836,-0.008292,0.002096,-0.031477,0.001616,-0.009167,0.002330,-0.023662,-0.025416
domestic,0.043370,-0.069274,1.000000,-0.041821,-0.038657,-0.050101,0.072056,-0.075669,0.004518,0.002332,0.002825,0.002082,0.004467,0.003772
beat,-0.035996,-0.015993,-0.041821,1.000000,0.939092,0.635785,-0.506381,0.612650,-0.473687,-0.003126,-0.000468,0.000737,-0.002075,-0.002319
district,-0.004956,-0.016780,-0.038657,0.939092,1.000000,0.688740,-0.499337,0.620597,-0.528367,-0.001122,-0.000027,0.000919,-0.001339,-0.001220
ward,0.013127,-0.015836,-0.050101,0.635785,0.688740,1.000000,-0.532559,0.626385,-0.432463,0.000059,-0.000011,0.001221,-0.000049,0.000588
communityarea,0.004968,-0.008292,0.072056,-0.506381,-0.499337,-0.532559,1.000000,-0.747118,0.240317,0.000821,0.001185,-0.000435,0.001802,0.001377
latitude,-0.005265,0.002096,-0.075669,0.612650,0.620597,0.626385,-0.747118,1.000000,-0.410834,-0.000028,-0.000483,0.001649,-0.003313,-0.002630
longitude,0.001056,-0.031477,0.004518,-0.473687,-0.528367,-0.432463,0.240317,-0.410834,1.000000,-0.002323,0.000345,-0.002221,0.007822,0.007829
awnd,0.036246,0.001616,0.002332,-0.003126,-0.001122,0.000059,0.000821,-0.000028,-0.002323,1.000000,0.080271,0.099045,-0.250733,-0.215913


In [5]:
df = pd.concat([df, pd.get_dummies(df['primarytype'])], axis = 1)

In [6]:
df = df.drop('primarytype', axis = 1)

In [7]:
#df = pd.concat([df, pd.get_dummies(df['description'])], axis = 1)

In [8]:
df = df.drop('description', axis = 1)

In [9]:
df = pd.concat([df, pd.get_dummies(df['locationdescription'])], axis = 1)

KeyboardInterrupt: 

In [ ]:
df = df.drop('fbicode', axis = 1)

In [ ]:
df = df.drop('locationdescription', axis = 1)

## Create a Model

In [38]:
#df.to_feather('chicago_crime_model_data.feather')

In [3]:
df = feather.read_dataframe('chicago_crime_model_data.feather')

In [4]:
df = df[0:1000000]

In [5]:
df.shape

(1000000, 228)

In [13]:
df.columns

Index(['index', 'arrest', 'domestic', 'beat', 'district', 'ward',
       'communityarea', 'latitude', 'longitude', 'awnd',
       ...
       'VEHICLE - OTHER RIDE SHARE SERVICE (E.G., UBER, LYFT)',
       'VEHICLE NON-COMMERCIAL', 'VEHICLE-COMMERCIAL',
       'VEHICLE-COMMERCIAL - ENTERTAINMENT/PARTY BUS',
       'VEHICLE-COMMERCIAL - TROLLEY BUS', 'VESTIBULE', 'WAREHOUSE',
       'WOODED AREA', 'YARD', 'YMCA'],
      dtype='object', length=228)

In [17]:
df = df[['index', 'arrest', 'domestic', 'beat']]

In [18]:
df.head()

,index,arrest,domestic,beat
0,1,0,0,511
1,2,0,0,512
2,3,0,0,1013
3,4,1,0,813
4,5,0,0,1033


In [19]:
y = df['arrest']
X = df.drop('arrest', axis = 1)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Logistic Regression

Need to standardize!!!!!

In [10]:
scaler = StandardScaler()

In [11]:
X_train_scaled = scaler.fit_transform(X_train)

In [12]:
X_test_scaled = scaler.transform(X_test)

In [13]:
clf_log = LogisticRegression().fit(X_train_scaled, y_train)

In [14]:
clf_log.score(X_test_scaled, y_test)

0.8599466666666666

In [15]:
preds_log = clf_log.predict(X_test)

In [16]:
roc_auc_score(y_test, preds_log)

0.5002236680001253

### Random Forest

In [21]:
len(df.columns)

4

In [24]:
clf = RandomForestClassifier(oob_score=True, n_jobs=-1).fit(X_train, y_train)

In [32]:
X_train

,index,domestic,beat
506532,510683,0,1312
327991,329990,0,1022
715937,728120,0,2533
860726,876361,0,1532
78617,79071,0,813
394612,396994,0,314
609887,619573,0,1233
471099,473880,0,2432
80595,81058,1,1032
995080,1013970,0,912


In [25]:
train_preds = clf.predict(X_train)

In [30]:
pd.to_pickle(clf, "pkl_simple.p")

In [27]:
pickle.dump(clf, open("pkl_simple.p"), "wb")

FileNotFoundError: [Errno 2] No such file or directory: 'pkl_simple.p'

In [6]:
clf.oob_score_

0.8633946524930664

In [8]:
preds = clf.predict(X_test)

In [9]:
roc_auc_score(y_test, preds)

0.8036966665451021

In [29]:
clf.score(X_test, y_test)

0.8734265038681394

In [30]:
preds = clf.predict(X_test)

In [31]:
roc_auc_score(y_test, preds)

0.7983847439909436

kernel SVM, KNN, Extra Trees